In [7]:
import torch
import pandas as pd 
import torchvision
from torchvision import datasets, transforms
from torch import nn,optim
import torch.nn.functional as F



df_train_ap = pd.read_csv('file://localhost/C:/Users/Bryan/Documents/School/Y3/FYP/UJIndoorLoc/new/df_train_ap.csv')
df_train_main = pd.read_csv('file://localhost/C:/Users/Bryan/Documents/School/Y3/FYP/UJIndoorLoc/new/df_train_main.csv')

df_test_ap = pd.read_csv('file://localhost/C:/Users/Bryan/Documents/School/Y3/FYP/UJIndoorLoc/new/df_test_ap.csv')
df_test_main = pd.read_csv('file://localhost/C:/Users/Bryan/Documents/School/Y3/FYP/UJIndoorLoc/new/df_test_main.csv')

In [8]:
label = df_train_main[["x","y"]]




#convert df to tensor object
train_t = torch.tensor(df_train_ap.values)

label_t = torch.tensor(label.values)

#flatten 
train_t = torch.flatten(train_t,1)

label_t = torch.flatten(label_t,1)
print(label_t.shape)
print(train_t.shape)
# print(label_t[0])
# print(b1f1_t[0])


train_data = [] 
for i in range(len(train_t)):
    train_data.append([train_t[i],label_t[i]])

trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=100)

print(len(trainloader))

torch.Size([13925, 2])
torch.Size([13925, 520])
140


In [9]:
testlabel = df_test_main[["x","y"]]

#df_test_ap = df_test_ap.replace([100],-110)

#df_test_ap = df_test_ap.apply(lambda x: (x-x.min())/(x.max()-x.min()), axis=1)


#convert df to tensor object
test_t = torch.tensor(df_test_ap.values)

testlabel_t = torch.tensor(testlabel.values)

#flatten 
test_t = torch.flatten(test_t,1)

testlabel_t = torch.flatten(testlabel_t,1)
print(testlabel_t.shape)
print(test_t.shape)
# print(label_t[0])
# print(b1f1_t[0])



testloader = torch.utils.data.DataLoader([ [test_t[i], testlabel_t[i]] for i in range(len(testlabel_t))], shuffle=True, batch_size=100)


torch.Size([6012, 2])
torch.Size([6012, 520])


In [10]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(520,500)
        self.fc2 = nn.Linear(500,500)
        self.fc3 = nn.Linear(500,500)
        self.fc4 = nn.Linear(500,2)
        
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self,x):
        #print(x[0][0])
        x = x.view(x.shape[0],-1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.fc4(x)
        
        return x

In [12]:
model = Classifier()

optimizer = optim.Adam(model.parameters(), lr=0.001, eps = 1e-9) 
#optimizer = optim.SGD(model.parameters(), lr = 0.001, weight_decay=0.03, momentum=0.005)

criterion = nn.MSELoss() #MSE
#criterion = torch.nn.L1Loss()

epochs = 50
train_losses, test_losses = [],[]

for e in range(epochs):
    tot_train_loss = 0 
    for data, labels in trainloader:
        data = data.view(data.shape[0],-1)
        data = data.float()
        optimizer.zero_grad()
        output = model(data)
        #output = ((output - torch.mean(output))/torch.max(output)-torch.min(output))
        loss = torch.sqrt(criterion(output,labels.float())) #RMSE 
        #loss = criterion(output,labels.float())
        tot_train_loss+=loss.item()

        loss.backward()
        optimizer.step()
        

    else:
        tot_test_loss = 0 
        with torch.no_grad():
            for data, labels in testloader:
                data = data.view(data.shape[0],-1)
                data=data.float()
                test_output = model(data)
                loss = torch.sqrt(criterion(test_output,labels.float()))
                #loss = criterion(test_output,labels.float())

                tot_test_loss+=loss.item()
                
        train_loss = tot_train_loss/len(trainloader.dataset)
        test_loss = tot_test_loss/len(trainloader.dataset)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
 
        print("Epoch:{}/{}..".format(e+1,epochs),
             "Training Loss:{:.3f}..".format(train_loss),
             "Test Loss:{:.3f}..".format(test_loss))



Epoch:1/50.. Training Loss:1.087.. Test Loss:0.446..
Epoch:2/50.. Training Loss:1.024.. Test Loss:0.460..
Epoch:3/50.. Training Loss:0.950.. Test Loss:0.388..
Epoch:4/50.. Training Loss:0.861.. Test Loss:0.379..
Epoch:5/50.. Training Loss:0.859.. Test Loss:0.365..
Epoch:6/50.. Training Loss:0.830.. Test Loss:0.362..
Epoch:7/50.. Training Loss:0.884.. Test Loss:0.372..
Epoch:8/50.. Training Loss:0.846.. Test Loss:0.369..
Epoch:9/50.. Training Loss:0.834.. Test Loss:0.379..
Epoch:10/50.. Training Loss:0.799.. Test Loss:0.358..
Epoch:11/50.. Training Loss:0.823.. Test Loss:0.386..
Epoch:12/50.. Training Loss:0.833.. Test Loss:0.354..
Epoch:13/50.. Training Loss:0.790.. Test Loss:0.369..
Epoch:14/50.. Training Loss:0.771.. Test Loss:0.320..
Epoch:15/50.. Training Loss:0.769.. Test Loss:0.320..
Epoch:16/50.. Training Loss:0.743.. Test Loss:0.329..
Epoch:17/50.. Training Loss:0.742.. Test Loss:0.322..
Epoch:18/50.. Training Loss:0.728.. Test Loss:0.311..
Epoch:19/50.. Training Loss:0.729.. T

In [ ]:
model = Classifier()

optimizer = optim.Adam(model.parameters(), lr=0.00955, eps = 1e-9) 
#optimizer = optim.SGD(model.parameters(), lr = 0.001, weight_decay=0.03, momentum=0.005)

criterion = nn.MSELoss() #MSE
#criterion = torch.nn.L1Loss()

epochs = 100

for e in range(epochs):
    running_loss = 0 
    for data, labels in trainloader:
        data = data.view(data.shape[0],-1)
        data = data.float()
        optimizer.zero_grad()
        output = model(data)
        #output = ((output - torch.mean(output))/torch.max(output)-torch.min(output))

        loss = torch.sqrt(criterion(output,labels.float())) #RMSE 
        #loss = criterion(labels.float(),output)

        loss.backward()
        optimizer.step()
        
        running_loss+=loss.item()
        

    else:
        print("labels x: ",(labels[0][0].float()),"label y: ", (labels[0][1].float()))
        print("output x: ",(output[0][0].float()),"output y: ", (output[0][1].float()))
        print("x DIFF: ",(labels[0][0].float()-output[0][0].float()).abs())
        print("y DIFF: ",(labels[0][1].float()-output[0][1].float()).abs())
        print(f"Training loss: {running_loss/len(trainloader)}")
        print()